In [1]:
devtools::install_github("clairbarnes/rwwa")
library(rwwa)

Skipping install of 'rwwa' from a github remote, the SHA1 (d8887905) has not changed since last install.
  Use `force = TRUE` to force installation

Loading required package: extRemes

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot




# Trend fitting for climate models

## CORDEX

_GMST only_

In [14]:
for (fnm in list.files("ts", pattern = "wetseason_.AM-[0-9]{2}", full.names = T)) {
    
    res_fnm <- gsub(".dat", ".csv", gsub("ts/", "res/", fnm))
    if(!file.exists(res_fnm)) {
        ts <- load_ts(fnm, col.names = c("year", "pr"))
        gmst <- load_ts(list.files("ts", pattern = paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4]), full.names = T), col.names = c("year", "gmst"))

        df <- merge(ts, gmst)

        cov_2023 <- df[df$year == 2023, "gmst", drop = F]
        cov_hist <- df[df$year == 2023, "gmst", drop = F] - 1.2
        cov_fut <- df[df$year == 2023, "gmst", drop = F] + 0.8

        mdl <- fit_ns("norm", "fixeddisp", df, "pr", "gmst", lower = T)

        res <- cmodel_results(mdl, rp = 10, cov_f = cov_2023, cov_hist = cov_hist, cov_fut = cov_fut,
                             y_now = 2023, y_start = 1979, y_fut = 2050, nsamp = 500)

        write.csv(res, res_fnm)
    }
}